<a href="https://colab.research.google.com/github/Humzafazal72/Recommendation-Systems/blob/main/Content_Filtering_Article_Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'article-recommendation-system:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F2502835%2F4247346%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240227%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240227T151651Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D3676d2d95a2cc1012ed7c70ca53ef859df56d54f2635a363546b88a9708e0ee9d21319125895628bb6163cbde228d8e54a691b2877e61ece1ead8f5b1856bdfe60fe2d3b73de6a636fd3d801c8358ca25ba7d469b186e379752777ec2ca2cf0119aed426c6d43850793fd99ea65f76c15f89987e2db74e9a051ee91f640bca124ec036435b1bd9dfc54764a7812942be7aed905da9adc2205bacf3d48cfa2a0ca9a6064c80cf15f9384c168da22426a0ce86edbbc55885ad8eb70524a7aff0dbdf89028431f00b56ce7dc922cb345f26a92342d0231720942186657d151c85b90b21e0e4a3ec398b8a7b7ee3a447f6dbf8cf6b838ce83dc3387476af7add934c'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
import joblib
import string
import nltk
from nltk.stem import LancasterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
ds=pd.read_csv('/kaggle/input/article-recommendation-system/articles.csv',encoding='latin1')

In [ ]:
ds.drop_duplicates()

,Article,Title
0,Data analysis is the process of inspecting and...,Best Books to Learn Data Analysis
1,The performance of a machine learning algorith...,Assumptions of Machine Learning Algorithms
2,You must have seen the news divided into categ...,News Classification with Machine Learning
3,When there are only two classes in a classific...,Multiclass Classification Algorithms in Machin...
4,The Multinomial Naive Bayes is one of the vari...,Multinomial Naive Bayes in Machine Learning
6,Natural language processing or NLP is a subfie...,Best Books to Learn NLP
7,By using a third-party application or API to m...,Send Instagram Messages using Python
8,Twitter is one of the most popular social medi...,Pfizer Vaccine Sentiment Analysis using Python
9,The squid game is currently one of the most tr...,Squid Game Sentiment Analysis using Python
10,Computer Vision is one of the fields of Artifi...,Best Books to Learn Computer Vision


In [ ]:
ls=list(np.arange(0,ds.shape[0],1))
id_df = pd.DataFrame({'id': ls})

In [ ]:
ds = pd.concat([ds, id_df], axis=1)

In [ ]:
punctuation=list(string.punctuation)
stop=set(stopwords.words('english'))

In [ ]:
stop.add('one')
stop.add('im')

In [ ]:
def cleaning(article):
    article=article.lower()
    stemmer=LancasterStemmer()
    no_punc = []
    words = article.split()
    clean=[]
    for word in words:
        clean_word = ''.join(char for char in word if char not in punctuation)
        no_punc.append(clean_word)
    for word in no_punc:
        if word not in stop:
            clean.append(word)
    clean_stem=[stemmer.stem(word) for word in clean]
    return " ".join(clean_stem)


In [ ]:
print(ds['Article'][13])
cleaned=cleaning(ds['Article'][13])
print(cleaned)

A voice recorder is found in every smartphone and computer today. It is an application that is used to record sound and save it in a specific file format, which can be listened to and transferred to another device. If you want to know how to make a voice recorder using Python, this article is for you. In this article, I will walk you through how to make a voice recorder using Python.
voic record found every smartphon comput today apply us record sound sav spec fil form list transfer anoth dev want know mak voic record us python artic artic walk mak voic record us python


In [ ]:
ds['Article']=ds['Article'].apply(cleaning)

In [ ]:
vectorizer=CountVectorizer()
vectorized=vectorizer.fit_transform(ds['Article'])

In [ ]:
joblib.dump(vectorizer, '/kaggle/working/count_vectorizer.pkl')

In [ ]:
cos_sim_matrix = cosine_similarity(vectorized)

In [ ]:
def similar(article_id=None,arr=None):
    if (article_id is not None and arr is not None) or (article_id is None and arr is None):
        print("error")
    elif article_id is not None:
        article=cos_sim_matrix[article_id]
    elif arr is not None:
        article = arr[0]
    sorted_indices = np.argsort(article)[::-1]
    most_similar=[(article[i], i) for i in sorted_indices]
    similar_ls=[]
    for i in range(1,6):
        article_id=most_similar[i][1]
        title = ds.loc[ds['id'] == article_id, 'Title'].values[0]
        similar_ls.append(title)
    return similar_ls

In [ ]:
print('Article Name',ds.loc[ds['id'] == 11, 'Title'].values)
print('Most similar articles:')
print(similar(11))

Article Name ['Best Resources to Learn Python']
Most similar articles:
['For Loop Over Keys and Values in a Python Dictionary', 'Best Books to Learn Data Analysis', 'Language Detection with Machine Learning', 'Best Python Frameworks to Build APIs', 'Swap Items of a Python List']
